<a href="https://colab.research.google.com/github/juba/StableDiffusionNotebook/blob/main/Stable_Diffusion_custom_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion custom notebook

Sources : 

- Stable Diffusion Github repo : https://github.com/CompVis/stable-diffusion
- Stable Diffusion HugginfaceHub page :https://huggingface.co/CompVis/stable-diffusion-v1-4
- Sample Hugginface diffusers notebook : https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

Ressources :

- Training set exploration : https://rom1504.github.io/clip-retrieval/

# 1. Setup

In [ ]:
#@title 1.1 Check allocated GPU
!nvidia-smi

In [ ]:
#@title 1.2 Install Python dependencies
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

In [ ]:
#@title 1.3 Google Drive
import os
import json
from glob import glob

use_google_drive = True #@param{type: 'boolean'}

if use_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  root_path = '/content/drive/MyDrive/AI/Stable_Diffusion'
  # Output dir
  modelPath = f'{root_path}/models'
  os.makedirs(modelPath, exist_ok=True)
  # Output dir
  outDirPath = f'{root_path}/images_out'
  os.makedirs(outDirPath, exist_ok=True)
  # Index for file names
  file_count = len(glob(outDirPath+"/*.txt")) + 1


In [ ]:
#@title 1.4 Downloading model
import torch
from diffusers import StableDiffusionPipeline

use_float16_precision = False#@param {type:"boolean"}
load_model_from_google_drive = True #@param{type: 'boolean'}
save_model_to_google_drive = False #@param{type: 'boolean'}
model_subdir = "float16" if use_float16_precision else "float32"

if not(load_model_from_google_drive):
  # Load model from Huggingface

  from google.colab import output
  output.enable_custom_widget_manager()

  from huggingface_hub import notebook_login
  notebook_login()

In [ ]:
#@title
if not(load_model_from_google_drive):
  print(f"Loading model from Huggingface")
  if use_float16_precision:
    pipe = StableDiffusionPipeline.from_pretrained(
      "CompVis/stable-diffusion-v1-4", 
      revision="fp16", torch_dtype=torch.float16, 
      use_auth_token=True
    )
  else:
    pipe = StableDiffusionPipeline.from_pretrained(
      "CompVis/stable-diffusion-v1-4", 
      use_auth_token=True
    )
  # Optionnaly save it to Drive
  if save_model_to_google_drive:
    print(f"Saving model to {modelPath}/{model_subdir}/")
    pipe.save_pretrained(f"{modelPath}/{model_subdir}/")

else:
  print(f"Loading model from {modelPath}/{model_subdir}/")
  pipe = StableDiffusionPipeline.from_pretrained(f"{modelPath}/{model_subdir}/")

In [ ]:
#@title 1.5 Utility code and functions

from PIL import Image
from torch import autocast
import random

# Move pipeline to GPU
pipe = pipe.to("cuda")

# Image grid utility
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


# Do diffusion
def do_diffuse(pipe, params):
  
  num_images = params['images_rows'] * params['images_cols']
  prompt = [params['prompt']] * num_images

  seed = params['seed']
  if seed == -1:
    random.seed()
    seed = random.randint(0, 2**32)

  generator = torch.Generator("cuda").manual_seed(seed)

  with autocast("cuda"):
    out = pipe(
        prompt,
        num_inference_steps=params['num_inference_steps'],
        guidance_scale=params['guidance_scale'],
        width=params['width'],
        height=params['height'],
        generator=generator
    )["sample"]
  
  if num_images > 1:
    out = image_grid(out, rows=params['images_rows'], cols=params['images_cols'])
  else:
    out = out[0]

  return (out, seed)



# 2. Generate images


In [ ]:
#@title 2.1 Settings

prompt = "A badass onion superhero" #@param {type: 'string'}
num_inference_steps = 50 #@param{type: 'number'}
guidance_scale = 8 #@param{type: 'number'}
width = 512 #@param{type: 'number'}
height = 512 #@param{type: 'number'}
images_rows = 1 #@param{type: 'number'}
images_cols = 1 #@param{type: 'number'}
seed = -1 #@param{type: 'number'}

params = {
    'prompt': prompt,
    'num_inference_steps': num_inference_steps,
    'guidance_scale': guidance_scale,
    'width': width,
    'height': height,
    'images_rows': images_rows,
    'images_cols': images_cols,
    'use_float16': use_float16_precision,
    'seed': seed
}

In [ ]:
#@title 2.2 Diffuse

image, seed = do_diffuse(pipe, params)

if use_google_drive:
  # Increment index
  file_count += 1
  # Save settings
  seed_orig = params['seed']
  params['seed'] = seed
  with open(f"{outDirPath}/img{file_count}_settings.txt", "w+") as f:   #save settings
        json.dump(params, f, ensure_ascii=False, indent=4)
  # Save image
  image.save(f"{outDirPath}/img{file_count}.jpg")
  params['seed'] = seed_orig
  

print(f"Using seed : {seed}")
display(image)